In [ ]:
import subprocess
import os
import signal

# Start the usb_cam node in the background
usb_cam_process = subprocess.Popen(
    ['ros2', 'run', 'usb_cam', 'usb_cam_node_exe'],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    preexec_fn=os.setsid # esto es clave para agrupar procesos
)

print("usb_cam node started.")

In [ ]:
# ROS 2 Camera Viewer in Jupyter (ROS 2 Foxy + usb_cam)

import rclpy
from rclpy.node import Node
from sensor_msgs.msg import Image
from cv_bridge import CvBridge
import cv2
from PIL import Image as PILImage
from IPython.display import display, clear_output
import io
import time

# Initialize the CV bridge
bridge = CvBridge()

# Create a subscriber node
class CameraSubscriber(Node):
    def __init__(self):
        super().__init__('camera_sub')
        self.subscription = self.create_subscription(
            Image,
            '/image_raw',  # Use your camera topic (e.g., /usb_cam/image_raw)
            self.listener_callback,
            10
        )

    def listener_callback(self, msg):
        # Convert ROS2 Image msg to OpenCV image
        frame = bridge.imgmsg_to_cv2(msg, desired_encoding='bgr8')

        # Convert to displayable format for Jupyter
        _, buffer = cv2.imencode('.jpg', frame)
        img_bytes = io.BytesIO(buffer)
        pil_img = PILImage.open(img_bytes)

        clear_output(wait=True)
        display(pil_img)

# Initialize ROS
rclpy.init()
node = CameraSubscriber()

# Receive and display a few frames
print("Receiving camera frames...")
for _ in range(10):
    rclpy.spin_once(node, timeout_sec=0.2)
    time.sleep(0.1)

# Clean up
node.destroy_node()
rclpy.shutdown()
print("Done.")


In [ ]:
# Matar todo el grupo de procesos lanzado por usb_cam
os.killpg(os.getpgid(usb_cam_process.pid), signal.SIGINT)

print("usb_cam node stopped.")